<a href="https://colab.research.google.com/github/Hoss3inf/EmojiPredictionNotebook/blob/master/emoji_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

دیتاست را دانلود می کنیم. شامل یک فایل فشرده با حدود ۴۱۲ هزار توییت در قالب ۲۰ فولدر


In [1]:
!wget http://51.77.107.71:8001/labels.rar

--2021-01-22 15:07:34--  http://51.77.107.71:8001/labels.rar
Connecting to 51.77.107.71:8001... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51981770 (50M) [application/x-rar-compressed]
Saving to: ‘labels.rar’

labels.rar          100%[===================>]  49.57M  5.05MB/s    in 10s     

2021-01-22 15:07:45 (4.75 MB/s) - ‘labels.rar’ saved [51981770/51981770]



از حالت فشرده خارج می کنیم. پس از اجرای این دستور از منوی سمت چپ فولدر
labels
قابل مشاهده است که شامل ۲۰ پوشه است.

هر پوشه حاوی توییت های مرتبط با یک نوع ایموجی است

In [ ]:
!unrar x labels.rar

Streaming output truncated to the last 5000 lines.
Extracting  labels/9/330467.txt                                           98%  OK 
Extracting  labels/9/330483.txt                                           98%  OK 
Extracting  labels/9/330490.txt                                           98%  OK 
Extracting  labels/9/330508.txt                                           98%  OK 
Extracting  labels/9/330530.txt                                           98%  OK 
Extracting  labels/9/330580.txt                                           98%  OK 
Extracting  labels/9/330583.txt                                           98%  OK 
Extracting  labels/9/330600.txt                                           98%  OK 
Extracting  labels/9/33063.txt                                            98%  OK 
Extracting  labels/9/330703.txt                                           98%  OK 
Extracting  labels/9/33071.tx

کتابخانه تنسورفلو را وارد می‌کنیم

In [ ]:
import tensorflow as tf

دیتاست 
train 
را وارد تنسورفلو می کنیم

In [ ]:
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'labels', 
    validation_split=0.1,
    seed = 5,
    subset='training')

Found 412261 files belonging to 20 classes.
Using 371035 files for training.


یک مثال از دیتاست وارد شده را می بینیم.

پنج نمونه از دیتاستی که وارد کرده ایم

In [ ]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(5):
    print("Review", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

Review b'"#haha #zerofucksgiven #zerof\'sgiven #subwaysign @ New York City Subway "'
Label 12
Review b"'| eating whoppers never looked so cool \\u2022 double date with @user + @user |\\u2026 '"
Label 16
Review b"'#Republique #kouignamann - so flaky and buttery #pastry #foodgasm #losangeles #brunch @user '"
Label 1
Review b"'Cheers lady! We made it @ Broad Ripple, Indiana '"
Label 13
Review b"'Uniquitiques will be closed today! Enjoy the snow and be safe! \\ufe0f #snow #thesouth #snow2016\\u2026 '"
Label 0


دیتاست 

validation

را که ده درصد دیتاست کل هست را وارد تنسورفلو می کنیم.

In [ ]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'labels', 
    validation_split=0.1, 
    subset='validation', 
    seed=5)

Found 412261 files belonging to 20 classes.
Using 41226 files for validation.


استانداردسازی دیتاست

همه کلمات را تبدیل به حروف کوچک می کنیم.

مثلا

Happy, happy, happY

سه حالت مختلف هستند که پس از استانداردسازی به یک حالت 

happy 

تبدیل می شوند

In [ ]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

کتابخانه ای که کلمات را تبدیل به وکتور می کند را وارد می کنیم

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

تعداد فیچر ها را ۱۰ هزار در نظر می گیریم.

ینی ده هزار کلمه متفاوت می توانیم داشته باشیم.

حداکثر طول هر توییت را هم ۲۵۰ در نظر می گیریم که بیش از حداکثر توییت یعنی ۱۴۰ کاراکتر است.

In [ ]:
max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

تعداد زیادی کتابخانه مرتبط را وارد می کنیم برای عملیات روی دیتاست

In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

لیبل ها را از دیتاست حذف می کنیم. این متغیر فقط شامل متن توییت هاست

In [ ]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [ ]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

یکی از توییت ها را تبدیل به وکتور می کنیم. کتابخانه تنسور، همه ی جمله ها را تبدیل به چنین برداری می کند

In [ ]:

text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[5], label_batch[5]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b"'lucky to have a life long friend like you \\ufe0f @ Gloucester City High School Announcements '", shape=(), dtype=string)
Label 0
Vectorized review (<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
array([[583,   6,  51,   8,  79, 226, 130,  55,  11,   4,   1,  61, 103,
         87,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0, 

برای مثال، چند کلمه و معادل برداری ان را می بینیم

In [ ]:
print("4 ---> ",vectorize_layer.get_vocabulary()[4])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

4 --->  ufe0f
 313 --->  tomorrow
Vocabulary size: 10000


این کد، دیتاست تست را برای تنسورفلو فراهم می کند

In [ ]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'labels', 
    batch_size=batch_size,
    seed=5)

Found 412261 files belonging to 20 classes.


In [ ]:
train_ds = raw_train_ds.map(vectorize_text)

این کد برای افزایش سرعت ترین کردن استفاده می شود که از تنظیمات پیش فرض استفاده شده

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
embedding_dim = 16

مدل و روش اموزش دیدن مدل را مشخص می کنیم

سپس خلاصه ای از مدل را می بینیم

In [ ]:
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(20)])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 16)          160016    
_________________________________________________________________
dropout_4 (Dropout)          (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                340       
Total params: 160,356
Trainable params: 160,356
Non-trainable params: 0
_________________________________________________________________


مدل را اصطلاخا کامپایل می کنیم تا برای آموزش دادن اماده شود. 

In [ ]:
model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.CategoricalAccuracy(
    name='categorical_accuracy', dtype=None))

مدل را با استفاده از دیتاست، اصطلاحا فیت می کنیم

In [ ]:
epochs = 10
history = model.fit(
    train_ds,
    epochs=epochs)

Epoch 1/10
11595/11595 [==============================] - 146s 13ms/step - loss: 2.7035 - categorical_accuracy: 0.9620
Epoch 2/10
11595/11595 [==============================] - 68s 6ms/step - loss: 2.3539 - categorical_accuracy: 0.3196
Epoch 3/10
11595/11595 [==============================] - 71s 6ms/step - loss: 2.1863 - categorical_accuracy: 0.2442
Epoch 4/10
11595/11595 [==============================] - 70s 6ms/step - loss: 2.1089 - categorical_accuracy: 0.2287
Epoch 5/10
11595/11595 [==============================] - 69s 6ms/step - loss: 2.0646 - categorical_accuracy: 0.2269
Epoch 6/10
11595/11595 [==============================] - 71s 6ms/step - loss: 2.0366 - categorical_accuracy: 0.2259
Epoch 7/10
11595/11595 [==============================] - 71s 6ms/step - loss: 2.0126 - categorical_accuracy: 0.2254
Epoch 8/10
11595/11595 [==============================] - 70s 6ms/step - loss: 1.9931 - categorical_accuracy: 0.2241
Epoch 9/10
11595/11595 [==============================] - 70s 

مدل را برای استفاده اصطلاحا استخراج می کنیم

In [ ]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])


In [ ]:
export_model.compile(
    loss=losses.CategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=['accuracy']
)



کتابخانه ایموجی را نصب می کنیم تا بتوانیم در ترمینال ایموجی ها را ببینیم

In [ ]:
!pip install emoji

     |████████████████████████████████| 51kB 3.0MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49717 sha256=d1ac5c4a56217c5e53eee0403e615608e658829743ccb66809cd9f7c9d40fb2d
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


اینجا یک جمله مثال می نویسیم و با مدلی که آموزش دادیم استفاده می کنیم تا ایموجی مناسب را پیش بینی کنیم

In [ ]:
examples = [
  "I'm Always the one getting get picked on - Best Birthday"
]

labels_probability_array = export_model.predict(examples)[0]

SyntaxError: ignored

کتابخانه ایموجی را وارد می کنیم

In [ ]:
import emoji as moji

احتمال اینکه جمله ی مثال بالا، متناسب هر کدام از ایموجی ها باشد را می توانیم اینجا ببینیم

هر چه احتمال به یک نزدیک تر باشد، یعنی ایموجی برای جمله متناسب تر است

In [ ]:
print(moji.emojize('1. Probability of ❤ is {}'.format(labels_probability_array[0])))
print(moji.emojize('2. Probability of :smiling_face_with_heart_eyes:  is {}',use_aliases=True).format(labels_probability_array[1]))
print(moji.emojize('3. Probability of :joy:  is {}'.format(labels_probability_array[2])))
print(moji.emojize('4. Probability of :two_hearts:  is {}'.format(labels_probability_array[3])))
print(moji.emojize('5. Probability of :fire:  is {}'.format(labels_probability_array[4])))
print(moji.emojize('6. Probability of 😊 is {}'.format(labels_probability_array[5])))
print(moji.emojize('7. Probability of :sunglasses:  is {}'.format(labels_probability_array[6])))
print(moji.emojize('8. Probability of :sparkles:  is {}'.format(labels_probability_array[7])))
print(moji.emojize('9. Probability of :blue_heart:  is {}'.format(labels_probability_array[8])))
print(moji.emojize('10. Probability of :kissing_heart:  is {}'.format(labels_probability_array[9])))
print(moji.emojize('11. Probability of ::camera::  is {}'.format(labels_probability_array[10])))
print(moji.emojize('12. Probability of :us:  is {}'.format(labels_probability_array[11])))
print(moji.emojize('13. Probability of :sunny:  is {}'.format(labels_probability_array[12])))
print(moji.emojize('14. Probability of ::purple_heart::  is {}'.format(labels_probability_array[13])))
print(moji.emojize('15. Probability of :wink:  is {}'.format(labels_probability_array[14])))
print(moji.emojize('16. Probability of :100:  is {}'.format(labels_probability_array[15])))
print(moji.emojize('17. Probability of :smile:  is {}'.format(labels_probability_array[16])))
print(moji.emojize('18. Probability of :evergreen_tree:  is {}'.format(labels_probability_array[17])))
print(moji.emojize('19. Probability of :camera:  is {}'.format(labels_probability_array[18])))
print(moji.emojize('20. Probability of :stuck_out_tongue_winking_eye:  is {}'.format(labels_probability_array[18])))

1. Probability of ❤  is 0.5568498969078064
2. Probability of :smiling_face_with_heart_eyes:  is 0.7482574582099915
3. Probability of :joy:  is 0.2178846299648285
4. Probability of 💕  is 0.11006885766983032
5. Probability of 🔥  is 0.04933905601501465
6. Probability of 😊  is 0.53814297914505
7. Probability of 🕶  is 0.6147952675819397
8. Probability of ✨  is 0.6382288932800293
9. Probability of 💙  is 0.5652240514755249
10. Probability of :kissing_heart:  is 0.11594727635383606
11. Probability of :📷:  is 0.21855106949806213
12. Probability of :us:  is 0.5905882120132446
13. Probability of :sunny:  is 0.8390136957168579
14. Probability of :💜:  is 0.7140940427780151
15. Probability of :wink:  is 0.3707720935344696
16. Probability of :100:  is 0.6809341311454773
17. Probability of :smile:  is 0.6406840682029724
18. Probability of 🌲  is 0.38163334131240845
19. Probability of 📷  is 0.5885104537010193
20. Probability of :stuck_out_tongue_winking_eye:  is 0.5885104537010193
